# Simple automatic differentiation illustration

In [1]:
from typing import Union, List

import numpy as np

np.set_printoptions(precision=4)

In [2]:
a = 3
a.__add__(4)

7

In [3]:
a = np.array([2,3,1,0])

print(a)
print("Addition using '__add__':", a.__add__(4))
print("Addition using '+':", a + 4)

[2 3 1 0]
Addition using '__add__': [6 7 5 4]
Addition using '+': [6 7 5 4]


In [4]:
Numberable = Union[float, int]

def ensure_number(num: Numberable):
    if isinstance(num, NumberWithGrad):
        return num
    else:
        return NumberWithGrad(num)        

class NumberWithGrad(object):
    
    def __init__(self, 
                 num: Numberable,
                 depends_on: List[Numberable] = None,
                 creation_op: str = ''):
        self.num = num
        self.grad = None
        self.depends_on = depends_on or []
        self.creation_op = creation_op

    def __add__(self, 
                other: Numberable):
        return NumberWithGrad(self.num + ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'add')
    
    def __mul__(self,
                other: Numberable = None):

        return NumberWithGrad(self.num * ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'mul')
    
    def backward(self, backward_grad: Numberable = None):
        if backward_grad is None: # first time calling backward
            self.grad = 1
        else: 
            # These lines allow gradients to accumulate.
            # If the gradient doesn't exist yet, simply set it equal
            # to backward_grad
            if self.grad is None:
                self.grad = backward_grad
            # Otherwise, simply add backward_grad to the existing gradient
            else:
                self.grad += backward_grad
        
        if self.creation_op == "add":
            # Simply send backward self.grad, since increasing either of these 
            # elements will increase the output by that same amount
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad)    

        if self.creation_op == "mul":

            # Calculate the derivative with respect to the first element
            new = self.depends_on[1] * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[0].backward(new.num)

            # Calculate the derivative with respect to the second element
            new = self.depends_on[0] * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[1].backward(new.num)

In [5]:
a = NumberWithGrad(3)
b = a * 4
c = b + 3
c.backward()
print(a.grad) # as expected
print(b.grad) # as expected

4
1


In [6]:
a = NumberWithGrad(3)

In [7]:
b = a * 4
c = b + 3
d = (a + 2)
e = c * d 
e.backward() 

In [8]:
a.grad # as expected

35